In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage import transform

tf.logging.set_verbosity(tf.logging.INFO)

# Load data
x_train = np.load('exam2_train_x.npy').astype(np.float32)
y_train = np.load('exam2_train_y.npy').astype(int)
# x_train = x_train/255.

x_test = np.load('exam2_test_x.npy').astype(np.float32)
y_test = np.load('exam2_test_y.npy').astype(int)
# x_test = x_test/255.

In [2]:
def gen_augmented_data(x, y):
    
    new_x_imgs = []
    new_y = []
    for x_train, y_train in zip(x, y):

        cw = transform.rotate(image=x_train, angle=15., mode='edge', resize=False, preserve_range=True)
        ccw = transform.rotate(image=x_train, angle=-15., mode='edge', resize=False, preserve_range=True)

        new_x_imgs.append(np.flipud(x_train))
        new_y.append(y_train)
        new_x_imgs.append(np.fliplr(x_train))
        new_y.append(y_train)
        new_x_imgs.append(cw)
        new_y.append(y_train)
        new_x_imgs.append(ccw)
        new_y.append(y_train)

    return new_x_imgs, new_y

In [5]:
def conv_model(features, mode, labels):
    input_layer = tf.reshape(features["x"], [-1, 64, 64, 3])
    
    c1 = tf.layers.conv2d(inputs=input_layer, filters=8, kernel_size=[4, 4], padding='same',
                          activation=tf.nn.relu, strides=2)
    p1 = tf.layers.max_pooling2d(inputs=c1, pool_size=[5, 5], strides=1)
    
    c2 = tf.layers.conv2d(inputs=p1, filters=16, kernel_size=[4, 4], padding='same',
                          activation=tf.nn.relu, strides=2)
    p2 = tf.layers.max_pooling2d(inputs=c2, pool_size=[5, 5], strides=1)
    
    c3 = tf.layers.conv2d(inputs=p2, filters=32, kernel_size=[4, 4], padding='same',
                          activation=tf.nn.relu, strides=1)
    p3 = tf.layers.max_pooling2d(inputs=c3, pool_size=[2, 2], strides=1)
    
    c4 = tf.layers.conv2d(inputs=p3, filters=64, kernel_size=[4, 4], padding='same',
                          activation=tf.nn.relu, strides=1)
    p4 = tf.layers.max_pooling2d(inputs=c4, pool_size=[2, 2], strides=1)
    
    print("p4 size:", p4.get_shape().as_list())
    
    a5 = tf.reshape(p4, [-1, 8*8*64])
    z5 = tf.layers.dense(a5, units=108, activation=tf.nn.relu)
    z5 = tf.layers.dropout(inputs=z5, rate=0.5, training=(mode == tf.estimator.ModeKeys.TRAIN))
    
    z6 = tf.layers.dense(z5, units=52, activation=tf.nn.relu)
    z6 = tf.layers.dropout(inputs=z6, rate=0.5, training=(mode == tf.estimator.ModeKeys.TRAIN))
    
    z7 = tf.layers.dense(z6, units=6, activation=tf.nn.sigmoid)

    predictions = {'classes': tf.argmax(input=z7, axis=1),
                   'probabilities': tf.nn.softmax(z7, name="softmax_tensor")}
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    onehot = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=6)

    loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot, logits=z7)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
        train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    eval_metric_ops = {"accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [7]:
tf.reset_default_graph()
tf.set_random_seed(5)

newx, newy = gen_augmented_data(x_train, y_train)
x_train = np.append(x_train, newx, axis=0).astype(np.float32)
y_train = np.append(y_train, newy, axis=0).astype(np.int)

x_train = x_train/255.

clf = tf.estimator.Estimator(model_fn=conv_model, model_dir='model')

log_hook = tf.train.LoggingTensorHook(tensors={"probabilities": "softmax_tensor"}, every_n_iter=100)

train_input = tf.estimator.inputs.numpy_input_fn(x={'x': x_train}, 
                                                 y=y_train, batch_size=85, 
                                                 num_epochs=None, shuffle=True)
clf.train(input_fn=train_input, steps=1, hooks=[log_hook])

In [105]:
predict_input = tf.estimator.inputs.numpy_input_fn(
    {'x': x_test}, y=y_test, num_epochs=1, shuffle=False)
results = clf.evaluate(input_fn=predict_input)

print(results)

INFO:tensorflow:Starting evaluation at 2018-04-14-22:32:40


INFO:tensorflow:Restoring parameters from .\model.ckpt-4101


INFO:tensorflow:Finished evaluation at 2018-04-14-22:32:41


INFO:tensorflow:Saving dict for global step 4101: accuracy = 0.15555556, global_step = 4101, loss = 1.7927731


{'accuracy': 0.15555556, 'loss': 1.7927731, 'global_step': 4101}
